# 타이타닉호 생존자 예측

- RMS 타이타닉 호(이하 타이타닉 호)는 1912년 4월 15일 빙산과 충돌하여 침몰한 북대서양 횡단 여객선이다.

- 타이타닉 호에 탑승했던 승객들의 승객번호, 선실등급, 이름, 성별, 나이, 배우자, 형제자매, 부모, 자녀 수, 티켓번호, 운임요금, 선실명, 승선장 정보가 원인변수, 생존여부가 결과변수로 들어있는 데이터프레임을 다양한 기계학습 기법(KNN, 로지스틱회귀분석, 나이브베이즈, SVM, 의사결정나무, 랜덤포레스트)을 이용해 생존여부를 예측 모델을 구축하고자 한다.

## 2. 데이터 전처리

### 1. 모듈 장착

In [2]:
# basic module
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re

# sklearn module
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import Lasso

### 2. 디렉토리 설정

In [3]:
os.chdir('C:/Users/ssmoo/Desktop/kaggle_titanic_prediction')
data_path = 'C:/Users/ssmoo/Desktop/kaggle_titanic_prediction/data/'

### 3. 데이터 불러오기

In [4]:
raw_train = pd.read_csv(data_path + 'raw_data/train.csv')
raw_test = pd.read_csv(data_path + 'raw_data/test.csv')

### 4. 데이터 복제

In [20]:
train = raw_train.copy()
test = raw_test.copy()

### 5. 데이터 확인

In [21]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


- 1장의 데이터 탐색에서 봤던 것과 마찬가지로 891개의 데이터와 Survived(생존여부) 변수를 포함한 12개의 변수가 들어있다.

In [22]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


- 테스트 데이터는 Survived(생존여부) 변수를 예측하여 Kaggle Competition 제출에 쓰일 데이터로 Survived(생존여부)변수를 제외한 11개의 변수와 418개의 데이터로 이루어져 있다.

### 6. 데이터 전처리(1) : 전처리를 위한 전처리

- 데이터를 전처리 하기 위해 함수를 만들 것이다.
- 이때 train 데이터프레임과 test 데이터프레임이 전처리 함수에 똑같이 적용될수 있도록 두 데이터의 서로 결측값이 다르게 존재하는 변수들을 채워주도록 하자.

##### train 데이터프레임 전처리

In [23]:
#%% train data preprocessing
embarked_notnull_idx = train['Embarked'].notnull()
train = train[embarked_notnull_idx]

- Embarked(승선장) 변수에 2개의 결측값이 존재한다. 이 결측값이 존재하는 행 데이터 2개를 지워 데이터 전처리를 돕는다.

##### test 데이터프레임 전처리

In [24]:
#%% test data preprocessing
PE = test[(test['Pclass'] == 3) & (test['Embarked'] == 'S')]
fare_median = np.median(PE[PE['Fare'].notnull()]['Fare'])
test['Fare'][152] = fare_median

C:\Users\ssmoo\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


- Fare(운임요금) 변수의 결측값이 존재한다. 이 데이터의 Pclass(선실등급)은 '3'값이며 Embarked(승선장)은 'S'값을 가지고 있다. 이러한 특징을 가지고 있는 데이터들의 평균 운임요금을 Fare(운임요금)의 결측값에 매워준다.

### 7. 데이터 전처리(2)

- train 데이터와 test데이터에 적용시킬 전처리 함수 코드를 구현한다.
- 전처리 코드는 1장(데이터탐색)을 기반으로 만들어졌다.
- Name 변수는 Mr, Mrs, Miss, Master의 주포지션으로 범주화 된 Position 변수로 변경하였다.
- SibSp, Parch 변수는 형제, 자매, 부모, 자식의 존재 여부에 따라 0, 1로 범주화하였다.
- 이상치가 많이 존재했던 Fare(운임요금) 변수는 robust scaling을 하였다.
- Age 변수의 결측값은 Survived(생존여부) 변수를 제외한 나머지 변수들로 lasso 회귀를 이용하여 예측하였다.
- Age 변수는 어린이, 청소년, 청년, 장년, 중년, 노년의 변수로 범주화 된 Aged 변수로 변경하였다.
- 범주형 변수는 one-hot encoding 처리 하였다.
- 코드에 주석으로 전처리 과정을 설명하였다.

In [26]:
def preprocessing(titanic_X):
    '''
    불필요한 변수 PassengerId 변수와
    결과변수인 Survived 변수를 제외한 요인변수 데이터를 입력받아 전처리 해준다.
    '''
    # 불필요한 titanic 변수 제거 (Ticket, Cabin)
    del titanic_X['Ticket'], titanic_X['Cabin']
    
    # 결측치 제거
    titanic_X = titanic_X[titanic_X['Embarked'].notnull()]
    
    # Name 변수 -> Position 변수로 변경
    def get_position(name):
        position_compile = re.compile('[A-z]*\.')
        position = position_compile.findall(name)
        
        return position[0][:-1]
    
    titanic_X['Position'] = titanic_X['Name'].apply(lambda name : get_position(name))
    del titanic_X['Name']
    
    # Name 변수값 Mr, Mrs, Miss, Master로 축약
    Mr_list = ['Capt', 'Col', 'Don', 'Jonkheer', 'Major', 'Rev', 'Sir']
    Miss_list = ['Countess', 'Lady', 'Mlle', 'Mme', 'Ms', 'Dona']
    
    for position in Mr_list:
        titanic_X.loc[titanic_X['Position'] == position, 'Position'] = 'Mr'
        
    for position in Miss_list:
        titanic_X.loc[titanic_X['Position'] == position, 'Position'] = 'Miss'
    
    titanic_X.loc[(titanic_X['Sex'] == 'male') & (titanic_X['Position'] == 'Dr'), 'Position'] = 'Mr'
    titanic_X.loc[(titanic_X['Sex'] == 'female') & (titanic_X['Position'] == 'Dr'), 'Position'] = 'Miss'
    
    list(set(titanic_X['Position']))
    
    # SibSp, Parch -> SibSp_existence, Parch_existence 변수로 변경
    def get_existence(num):
        if num == 0:
            return 0
        elif num >= 1:
            return 1
        else:
            return None
    
    titanic_X['SibSp_existence'] = titanic_X['SibSp'].apply(lambda num : get_existence(num))
    titanic_X['Parch_existence'] = titanic_X['Parch'].apply(lambda num : get_existence(num))
    
    del titanic_X['SibSp'], titanic_X['Parch']
    
    # Fare 변수 scaling
    robust = RobustScaler()
    titanic_X['Fare'] = robust.fit_transform(titanic_X[['Fare']])
    
    # one_hot encoding_1
    feature_list = ['Pclass', 'Embarked', 'Position', 'SibSp_existence', 'Parch_existence']
    for feature in feature_list:
        titanic_X[feature] = titanic_X[feature].astype(str)
    
    titanic_X = pd.get_dummies(titanic_X)
    
    # Age 변수 결측값 예측
    age_notnull = titanic_X[titanic_X['Age'].notnull()]
    age_isnull = titanic_X[titanic_X['Age'].isnull()]
    age_notnull_X = age_notnull.iloc[:, 1:]
    age_notnull_y = age_notnull.iloc[:, 0]
    age_isnull_X = age_isnull.iloc[:, 1:]
    lasso = Lasso(alpha = 0.01)
    lasso.fit(age_notnull_X.values, age_notnull_y.values)
    age_isnull['Age'] = lasso.predict(age_isnull_X.values)
    
    titanic_X = pd.concat([age_notnull, age_isnull]).sort_index()
    
    # Age 변수 -> Aged 변수로 변경
    def get_aged(age):
        if age < 13:
            return 0
        elif age < 20:
            return 1
        elif age < 30:
            return 2
        elif age < 45:
            return 3
        elif age < 60:
            return 4
        elif age >= 60:
            return 5
        else:
            return None
    
    titanic_X['Aged'] = titanic_X['Age'].apply(lambda age : get_aged(age))
    del titanic_X['Age']
    titanic_X['Aged'] = titanic_X['Aged'].astype(str)
    
    # one-hot encoding_2
    titanic_X = pd.get_dummies(titanic_X)
    
    return titanic_X

### 8. 전처리 데이터 생성

In [27]:
train_X = train.iloc[:, 2:]
train_X = preprocessing(train_X)
train_y = train.iloc[:, [1]]

test_X = test.iloc[:, 1:]
test_X = preprocessing(test_X)

C:\Users\ssmoo\Anaconda3\lib\site-packages\ipykernel_launcher.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\ssmoo\Anaconda3\lib\site-packages\ipykernel_launcher.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [28]:
train_X.head()

,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Position_Master,...,SibSp_existence_0,SibSp_existence_1,Parch_existence_0,Parch_existence_1,Aged_0,Aged_1,Aged_2,Aged_3,Aged_4,Aged_5
0,-0.311813,0,0,1,0,1,0,0,1,0,...,0,1,1,0,0,0,1,0,0,0
1,2.459687,1,0,0,1,0,1,0,0,0,...,0,1,1,0,0,0,0,1,0,0
2,-0.282598,0,0,1,1,0,0,0,1,0,...,1,0,1,0,0,0,1,0,0,0
3,1.672674,1,0,0,1,0,0,0,1,0,...,0,1,1,0,0,0,0,1,0,0
4,-0.277188,0,0,1,0,1,0,0,1,0,...,1,0,1,0,0,0,0,1,0,0


In [29]:
train_y.head()

,Survived
0,0
1,1
2,1
3,1
4,0


In [30]:
test_X.head()

,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Position_Master,...,SibSp_existence_0,SibSp_existence_1,Parch_existence_0,Parch_existence_1,Aged_0,Aged_1,Aged_2,Aged_3,Aged_4,Aged_5
0,-0.281005,0,0,1,0,1,0,1,0,0,...,1,0,1,0,0,0,0,1,0,0
1,-0.316176,0,0,1,1,0,0,0,1,0,...,0,1,1,0,0,0,0,0,1,0
2,-0.202184,0,1,0,0,1,0,1,0,0,...,1,0,1,0,0,0,0,0,0,1
3,-0.245660,0,0,1,0,1,0,0,1,0,...,1,0,1,0,0,0,1,0,0,0
4,-0.091902,0,0,1,1,0,0,0,1,0,...,0,1,0,1,0,0,1,0,0,0


- 전처리 결과 23개의 변수가 만들어 졌다.

### 9. 전처리 데이터 저장

In [ ]:
#%% save data
train_X.to_csv('train_X.csv', index = False)
train_y.to_csv('train_y.csv', index = False)
test_X.to_csv('test_X.csv', index = False)